In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pyarrow as pa

import os
import random

import matplotlib.gridspec as gridspec
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import statistics
from collections import Counter

import os
from collections import OrderedDict
import scipy.stats
import re
sns.set_style("whitegrid")


In [2]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

In [10]:
taskDF = pd.read_parquet(filePath)
taskDF['ParticipantNum'] = taskDF["ParticipantPrivateID"]
taskDF = taskDF.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
taskDF = taskDF[taskDF['ParticipantPrivateID'].notna()]
taskDF = taskDF[taskDF['ParticipantPrivateID']!='nan']
taskDF = taskDF.astype({"ParticipantNum": float})

print(len(taskDF.ParticipantPublicID.unique()))

245


In [11]:
pID = "5fc94c64c8a3e03ded162dd4"

tmp = taskDF[taskDF['ParticipantPublicID']==pID]

print(tmp[tmp['display']=="likerts"][['SpreadsheetRow', 'ScreenNumber', 'ScreenName', 'ZoneName', 'ZoneType', \
                                      'ResponseType', 'Response',  'Correct',  'display', 'Text1']])

    SpreadsheetRow  ScreenNumber ScreenName    ZoneName              ZoneType  \
57            28.0           1.0   Screen 4       Zone6  response_button_text   
58            29.0           1.0   Screen 4       Zone7  response_button_text   
59            30.0           1.0   Screen 4       Zone7  response_button_text   
60            31.0           1.0   Screen 4  NextButton  response_button_text   
86            42.0           1.0   Screen 4       Zone6  response_button_text   
87            43.0           1.0   Screen 4       Zone7  response_button_text   
88            44.0           1.0   Screen 4       Zone7  response_button_text   
89            45.0           1.0   Screen 4  NextButton  response_button_text   

    ResponseType       Response  Correct  display  \
57           NaN  Somewhat True      0.0  likerts   
58           NaN      Very True      0.0  likerts   
59           NaN      Very True      0.0  likerts   
60           NaN        Neutral      0.0  likerts   
86   

In [12]:
# randomiser_k74v = LVL
# randomiser_8rx4 = VisCond_DIFF
# randomiser_zcqq = VisCond_EASY

# randomiser_ndwh = AgentCond_BlurVis_DIFF
# randomiser_7vb5 = AgentCond_Blur_DIFF
# randomiser_fy7r = AgentCond_Vis_DIFF
# randomiser_vfim = AgentCond_Norm_DIFF

# randomiser_l6v8 = AgentCond_BlurVis_EASY
# randomiser_dwc2 = AgentCond_Blur_EASY
# randomiser_xnr7 = AgentCond_Vis_EASY
# randomiser_9z8b = AgentCond_Norm_EASY

taskDF.rename(columns={ "randomiser_k74v": "LVL", 
                        "randomiser_8rx4": "VisCond_DIFF", 
                        "randomiser_zcqq": "VisCond_EASY",
                        "randomiser_ndwh": "AgentCond_BlurVis_DIFF",
                        "randomiser_7vb5": "AgentCond_Blur_DIFF",
                        "randomiser_fy7r": "AgentCond_Vis_DIFF",
                        "randomiser_vfim": "AgentCond_Norm_DIFF",
                        "randomiser_l6v8": "AgentCond_BlurVis_EASY",
                        "randomiser_dwc2": "AgentCond_Blur_EASY",
                        "randomiser_xnr7": "AgentCond_Vis_EASY",
                        "randomiser_9z8b": "AgentCond_Norm_EASY",
                        "Q#": "Q_Num"}, inplace=True)

In [13]:
# Need to Use Spreadsheet to determine difficulty and visual condition

ls = taskDF['Spreadsheet'].unique()
new = taskDF['Spreadsheet'].str.split("_", n = 5, expand = True)
new[5] = new[5].str.replace('Blur','Blurred')
new[5] = new[5].str.replace('Vis','Visuals')
new[5] = new[5].str.replace('Blurred_and_Visuals','Blur_Plus_Vis')
new[5] = new[5].str.replace('Blurred_And_Visuals','Blur_Plus_Vis')
new[5] = new[5].str.replace('Normal','Video')
print(new)
taskDF['VisCond']=new[5]
print(taskDF.VisCond.unique())

         0    1   2   3    4              5
0      STD  300  VS  PP  700  Blur_Plus_Vis
1      STD  300  VS  PP  700  Blur_Plus_Vis
2      STD  300  VS  PP  700  Blur_Plus_Vis
3      STD  300  VS  PP  700  Blur_Plus_Vis
4      STD  300  VS  PP  700  Blur_Plus_Vis
...    ...  ...  ..  ..  ...            ...
75810  STD  300  VS  PP  700        Blurred
75811  STD  300  VS  PP  700        Blurred
75812  STD  300  VS  PP  700        Blurred
75813  STD  300  VS  PP  700        Blurred
75814  STD  300  VS  PP  700        Blurred

[75676 rows x 6 columns]
['Blur_Plus_Vis' 'Visuals' 'Blurred' 'Video']


In [14]:
lookup = taskDF[["AgentCond_BlurVis_DIFF","AgentCond_Blur_DIFF","AgentCond_Vis_DIFF","AgentCond_Norm_DIFF",
                 "AgentCond_BlurVis_EASY","AgentCond_Blur_EASY","AgentCond_Vis_EASY","AgentCond_Norm_EASY"]].notnull().idxmax(1)
taskDF = taskDF.assign(AgentCond=taskDF.lookup(lookup.index, lookup.values))
# taskDF['AgentCond']
print(taskDF.ExperimentVersion.unique())

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  This is separate from the ipykernel package so we can avoid doing imports until


[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]


In [15]:
filePath = os.path.join(tempTopDir, 'GP_Consent_Long.parquet')
consentDF_L = pd.read_parquet(filePath).infer_objects()
consentDF_L = consentDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_L = consentDF_L[consentDF_L['ParticipantPrivateID'].notna()]
consentDF_L = consentDF_L[consentDF_L['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_Consent_Short.parquet')
consentDF_S = pd.read_parquet(filePath).infer_objects()
consentDF_S = consentDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_S = consentDF_S[consentDF_S['ParticipantPrivateID'].notna()]
consentDF_S = consentDF_S[consentDF_S['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_Demographics_Long.parquet')
demoDF_L = pd.read_parquet(filePath).infer_objects()
demoDF_L = demoDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_L = demoDF_L[demoDF_L['ParticipantPrivateID'].notna()]
demoDF_L = demoDF_L[demoDF_L['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_Demographics_Short.parquet')
demoDF_S = pd.read_parquet(filePath).infer_objects()
demoDF_S = demoDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_S = demoDF_S[demoDF_S['ParticipantPrivateID'].notna()]
demoDF_S = demoDF_S[demoDF_S['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_AI_Long.parquet')
aiDF_L = pd.read_parquet(filePath).infer_objects()
aiDF_L = aiDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_L = aiDF_L[aiDF_L['ParticipantPrivateID'].notna()]
aiDF_L = aiDF_L[aiDF_L['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_AI_Short.parquet')
aiDF_S = pd.read_parquet(filePath).infer_objects()
aiDF_S = aiDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_S = aiDF_S[aiDF_S['ParticipantPrivateID'].notna()]
aiDF_S = aiDF_S[aiDF_S['ParticipantPrivateID']!='nan']

In [16]:
print(len(demoDF_L.ParticipantPrivateID.unique()))
print(demoDF_L.ExperimentVersion.unique())

245
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]


In [17]:
print(len(taskDF.ParticipantPrivateID.unique()))
print(taskDF.ExperimentVersion.unique())
print(aiDF_L.ExperimentVersion.unique())
print(consentDF_L.ExperimentVersion.unique())

245
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]


In [18]:
taskDF.ExperimentVersion.unique()

array([60., 62., 64., 65., 67., 70., 72., 73., 76., 77.])

In [19]:
# Check that the difficulty column (LVL) matches the spreadsheet column
difficulty = []
# df_str = taskDF.query('Q_Num==Q_Num & (ZoneType=="response_button_text" | ZoneType=="response_text_area")')
# df_str['Question'] =  [re.split('/|>', str(x))[4] for x in df_str['Q_Num']]

sheet = taskDF['Spreadsheet']
tmp_DF = pd.DataFrame(sheet.str.split(pat="_",expand=True))
print(tmp_DF)

taskDF['LVL'] = np.where((tmp_DF[1]=='23') & (tmp_DF[4]=='23'), 'Difficult',
                           np.where((tmp_DF[1]=='300') & (tmp_DF[4]=='700'), 'Difficult',
                                    np.where((tmp_DF[1]=='700') & (tmp_DF[4]=='300'),'Difficult','Easy')))

         0    1   2   3    4     5     6     7
0      STD  300  VS  PP  700  Blur   And   Vis
1      STD  300  VS  PP  700  Blur   And   Vis
2      STD  300  VS  PP  700  Blur   And   Vis
3      STD  300  VS  PP  700  Blur   And   Vis
4      STD  300  VS  PP  700  Blur   And   Vis
...    ...  ...  ..  ..  ...   ...   ...   ...
75810  STD  300  VS  PP  700  Blur  None  None
75811  STD  300  VS  PP  700  Blur  None  None
75812  STD  300  VS  PP  700  Blur  None  None
75813  STD  300  VS  PP  700  Blur  None  None
75814  STD  300  VS  PP  700  Blur  None  None

[75676 rows x 8 columns]


In [20]:
taskDF['AttnQ'] = None
# Assign agent order & attention question number in new columns
for p_index, pID in enumerate(taskDF.ParticipantPrivateID.unique()):
    temp = taskDF[taskDF['ParticipantPrivateID']==pID].copy()
    if (temp.ExperimentVersion.unique()[0]<76):
#         print(temp.ExperimentVersion.unique()[0])
        # set up column
        if temp.VisCond.unique()[0] == 'Blur_Plus_Vis':
            temp['AgentNum'] = np.where((temp['SpreadsheetRow'] >= 18) & (temp['SpreadsheetRow'] <= 31), 1,
                            np.where((temp['SpreadsheetRow'] >= 33) & (temp['SpreadsheetRow'] <= 45), 2,
                            np.where((temp['SpreadsheetRow'] >= 47) & (temp['SpreadsheetRow'] <= 62), 3, 0)))
        else:
            temp['AgentNum'] = np.where((temp['SpreadsheetRow'] >= 18) & (temp['SpreadsheetRow'] <= 30), 1,
                            np.where((temp['SpreadsheetRow'] >= 32) & (temp['SpreadsheetRow'] <= 43), 2,
                            np.where((temp['SpreadsheetRow'] >= 45) & (temp['SpreadsheetRow'] <= 60), 3, 0)))
            
        temp['AttnQ'] = np.where((temp['display']=="NextLvl") &(temp['ZoneType']=="response_button_text")
                                 &(temp['AgentNum']==1), 'A1',
                            np.where((temp['display']=="NextLvl") &(temp['ZoneType']=="response_button_text")
                                 &(temp['AgentNum']==2), 'A3',
                            np.where((temp['ANSWER']=="Using the tunnels on the sides of the maze ") 
                                     &(temp['ZoneType']=="response_button_text"),'A5',
                            np.where((temp['ANSWER']=="Black") &(temp['ZoneType']=="response_button_text")
                                     ,'A6', 0))))
        taskDF.update(temp[['AgentNum','AttnQ']])
#         print("\n............" + str(temp.VisCond.unique()[0]))
#         print(temp[temp['AgentNum']==3].Response.unique())
    # Now number attention qs for later versions
    else:
        temp['AttnQ'] = np.where((temp['display']=="Attn2") &(temp['ZoneType']=="response_button_text")
                                 &(temp['AgentNum']==1), 'A1',
                            np.where((temp['display']=="Attn1") &(temp['ZoneType']=="response_button_text")
                                 &(temp['AgentNum']==1), 'A2',
                            np.where((temp['display']=="Attn2") &(temp['ZoneType']=="response_button_text")
                                 &(temp['AgentNum']==2), 'A3',
                            np.where((temp['display']=="Attn1") &(temp['ZoneType']=="response_button_text")
                                 &(temp['AgentNum']==2), 'A4',
                            np.where((temp['ANSWER']=="Using the tunnels on the sides of the maze ") 
                                     &(temp['ZoneType']=="response_button_text"),'A5',
                            np.where((temp['ANSWER']=="Black") &(temp['ZoneType']=="response_button_text")
                                     ,'A6', 0))))))
        taskDF.update(temp['AttnQ'])
print(taskDF.AttnQ.unique())

['0' 'A1' 'A3' 'A5' 'A6' 'A2' 'A4']


In [21]:
pID = "5ea891c923a85e2eeb157452"

ls = taskDF[taskDF['ParticipantPublicID']==pID][['display','Response','ZoneType']].values.tolist()
print(taskDF[taskDF['ParticipantPublicID']==pID]['Spreadsheet'].unique())
for item in ls:
    if item[0] == "likerts":
        print(item)

['PP_23_VS_STD_23_Blur']
['likerts', 'VIDEO STARTED', 'content_video']
['likerts', 'VIDEO PLAYING EVENT FIRED', 'content_video']
['likerts', 'VIDEO TIMEUPDATE EVENT FIRED', 'content_video']
['likerts', 'ADJUSTED START TIME based on TIMEUPDATE EVENT', 'content_video']
['likerts', 'Somewhat False', 'response_button_text']
['likerts', 'VIDEO STARTED', 'content_video']
['likerts', 'VIDEO PLAYING EVENT FIRED', 'content_video']
['likerts', 'VIDEO TIMEUPDATE EVENT FIRED', 'content_video']
['likerts', 'ADJUSTED START TIME based on TIMEUPDATE EVENT', 'content_video']
['likerts', 'Somewhat False', 'response_button_text']
['likerts', 'VIDEO STARTED', 'content_video']
['likerts', 'VIDEO PLAYING EVENT FIRED', 'content_video']
['likerts', 'VIDEO TIMEUPDATE EVENT FIRED', 'content_video']
['likerts', 'ADJUSTED START TIME based on TIMEUPDATE EVENT', 'content_video']
['likerts', 'Somewhat True', 'response_button_text']
['likerts', 'VIDEO STARTED', 'content_video']
['likerts', 'VIDEO PLAYING EVENT FIRED'

In [22]:
# Number the Likert Responses for first agent
taskDF['Likert_Num']=0
for p_index, pID in enumerate(taskDF.ParticipantPrivateID.unique()):
    temp = taskDF[taskDF['ParticipantPrivateID']==pID].copy()
    tmp = temp.query('ZoneType=="response_button_text" &display=="likerts" & AgentNum==1').copy()
    tmp['Likert_Num'] = range(1, len(tmp)+1)
    if (tmp.VisCond.unique()[0]=="Blur_Plus_Vis" and len(tmp)<4):
        print(tmp[['ParticipantPublicID','AgentNum','ExperimentVersion','Spreadsheet']])
#     print(tmp.Question.unique())
#     print(tmp[['Question','display']])
    taskDF.update(tmp['Likert_Num'])
#     if (taskDF[taskDF['VisCond']=="Blur_Plus_Vis"].Likert_Num.value_counts()[4]>47):
    print(taskDF[taskDF['VisCond']=="Blur_Plus_Vis"].Likert_Num.value_counts())
    
# Number the Likert Responses for second agent
for p_index, pID in enumerate(taskDF.ParticipantPrivateID.unique()):
    temp = taskDF[taskDF['ParticipantPrivateID']==pID].copy()
    tmp = temp.query('ZoneType=="response_button_text" &display=="likerts" & AgentNum==2').copy()
    tmp['Likert_Num'] = range(1, len(tmp)+1)
    if (tmp.VisCond.unique()[0]=="Blur_Plus_Vis" and len(tmp)<4):
        print(tmp[['ParticipantPublicID','AgentNum','Spreadsheet']])
#     print(tmp.Question.unique())
#     print(tmp[['Question','display']])
    taskDF.update(tmp['Likert_Num'])
#     if (taskDF[taskDF['VisCond']=="Blur_Plus_Vis"].Likert_Num.value_counts()[4]>47):
    print(taskDF[taskDF['VisCond']=="Blur_Plus_Vis"].Likert_Num.value_counts())



0.0    16851
1.0        1
2.0        1
3.0        1
4.0        1
Name: Likert_Num, dtype: int64
0.0    16851
1.0        1
2.0        1
3.0        1
4.0        1
Name: Likert_Num, dtype: int64
0.0    16851
1.0        1
2.0        1
3.0        1
4.0        1
Name: Likert_Num, dtype: int64
0.0    16847
1.0        2
2.0        2
3.0        2
4.0        2
Name: Likert_Num, dtype: int64
0.0    16843
1.0        3
2.0        3
3.0        3
4.0        3
Name: Likert_Num, dtype: int64
0.0    16843
1.0        3
2.0        3
3.0        3
4.0        3
Name: Likert_Num, dtype: int64
0.0    16839
1.0        4
2.0        4
3.0        4
4.0        4
Name: Likert_Num, dtype: int64
0.0    16835
1.0        5
2.0        5
3.0        5
4.0        5
Name: Likert_Num, dtype: int64
0.0    16835
1.0        5
2.0        5
3.0        5
4.0        5
Name: Likert_Num, dtype: int64
0.0    16835
1.0        5
2.0        5
3.0        5
4.0        5
Name: Likert_Num, dtype: int64
0.0    16835
1.0        5
2.0        5
3

0.0    16763
1.0       23
2.0       23
3.0       23
4.0       23
Name: Likert_Num, dtype: int64
0.0    16763
1.0       23
2.0       23
3.0       23
4.0       23
Name: Likert_Num, dtype: int64
0.0    16763
1.0       23
2.0       23
3.0       23
4.0       23
Name: Likert_Num, dtype: int64
0.0    16763
1.0       23
2.0       23
3.0       23
4.0       23
Name: Likert_Num, dtype: int64
0.0    16759
1.0       24
2.0       24
3.0       24
4.0       24
Name: Likert_Num, dtype: int64
0.0    16759
1.0       24
2.0       24
3.0       24
4.0       24
Name: Likert_Num, dtype: int64
0.0    16759
1.0       24
2.0       24
3.0       24
4.0       24
Name: Likert_Num, dtype: int64
0.0    16759
1.0       24
2.0       24
3.0       24
4.0       24
Name: Likert_Num, dtype: int64
0.0    16755
1.0       25
2.0       25
3.0       25
4.0       25
Name: Likert_Num, dtype: int64
0.0    16751
1.0       26
2.0       26
3.0       26
4.0       26
Name: Likert_Num, dtype: int64
0.0    16747
1.0       27
2.0       27
3

0.0    16683
1.0       43
2.0       43
3.0       43
4.0       43
Name: Likert_Num, dtype: int64
0.0    16683
1.0       43
2.0       43
3.0       43
4.0       43
Name: Likert_Num, dtype: int64
0.0    16679
1.0       44
2.0       44
3.0       44
4.0       44
Name: Likert_Num, dtype: int64
0.0    16679
1.0       44
2.0       44
3.0       44
4.0       44
Name: Likert_Num, dtype: int64
0.0    16679
1.0       44
2.0       44
3.0       44
4.0       44
Name: Likert_Num, dtype: int64
0.0    16679
1.0       44
2.0       44
3.0       44
4.0       44
Name: Likert_Num, dtype: int64
0.0    16675
1.0       45
2.0       45
3.0       45
4.0       45
Name: Likert_Num, dtype: int64
0.0    16675
1.0       45
2.0       45
3.0       45
4.0       45
Name: Likert_Num, dtype: int64
0.0    16675
1.0       45
2.0       45
3.0       45
4.0       45
Name: Likert_Num, dtype: int64
0.0    16671
1.0       46
2.0       46
3.0       46
4.0       46
Name: Likert_Num, dtype: int64
0.0    16671
1.0       46
2.0       46
3

0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3.0       61
4.0       61
Name: Likert_Num, dtype: int64
0.0    16611
1.0       61
2.0       61
3

0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3.0       83
4.0       83
Name: Likert_Num, dtype: int64
0.0    16523
1.0       83
2.0       83
3

0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3.0      104
4.0      104
Name: Likert_Num, dtype: int64
0.0    16439
1.0      104
2.0      104
3

In [27]:
pID = "5fc94c64c8a3e03ded162dd4"

tmp = taskDF[taskDF['ParticipantPublicID']==pID]

print(tmp[tmp['display']=="likerts"][['SpreadsheetRow', 'ScreenNumber', 'ScreenName', 'ZoneName', 'ZoneType', \
                                      'ResponseType', 'Response',  'Correct',  'display', 'Text1']])

print(len(taskDF.ParticipantPublicID.unique()))

    SpreadsheetRow  ScreenNumber ScreenName    ZoneName              ZoneType  \
57            28.0           1.0   Screen 4       Zone6  response_button_text   
58            29.0           1.0   Screen 4       Zone7  response_button_text   
59            30.0           1.0   Screen 4       Zone7  response_button_text   
60            31.0           1.0   Screen 4  NextButton  response_button_text   
86            42.0           1.0   Screen 4       Zone6  response_button_text   
87            43.0           1.0   Screen 4       Zone7  response_button_text   
88            44.0           1.0   Screen 4       Zone7  response_button_text   
89            45.0           1.0   Screen 4  NextButton  response_button_text   

    ResponseType       Response  Correct  display  \
57           NaN  Somewhat True      0.0  likerts   
58           NaN      Very True      0.0  likerts   
59           NaN      Very True      0.0  likerts   
60           NaN        Neutral      0.0  likerts   
86   

In [26]:
tempDir = '/Volumes/Britt_SSD/ReDoEvals3/'
tempTopDir = os.path.join(tempDir, 'UserStudy')
if not os.path.isdir(tempTopDir):
    os.mkdir(tempTopDir)
tempTopDir = os.path.join(tempDir, 'UserStudy')

# Write Main Task DF to parquet:
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')
taskDF.to_parquet(path=filePath,compression='brotli')
# table = pa.Table.from_pandas(taskDF, safe=False)
# Parquet with Brotli compression
# pq.write_table(table, filePath, compression='BROTLI')
filePath = os.path.join(tempTopDir, 'GP_Main.csv')
taskDF.to_csv(filePath)